### Case Condition (Brief)

### Задание 1:  
***Имеется набор данных со следующими столбцами:***  
    1) Идентификатор документа  
    2) Идентификатор товара  
    3) Операция документа  
    4) Дата документа  
    5) Идентификатор филиала, создавшего документ  
    
> *Идентификатор товара – уникальный номер единицы товара, находящейся в сервисном центре*

Этот набор данных связан со справочником филиалов. Справочник содержит следующие поля:  
    1) Идентификатор филиала  
    2) Наименование филиала  
    3) Наименование региона филиала  
    
***Необходимо рассчитать среднее время нахождения товара в сервисном центре:   
    1) Общая динамика по месяцам   
    2) Динамика по неделям в разрезе регионов  
Вывести нормативный срок по всей компании.  
Графически представить динамику отклонения от нормы по каждому региону.***

### Задание 2:  
Вы работаете сотрудником, осуществляющим прием клиентов в сервисном центре.  
К вам пришел клиент.  
Какие варианты развития событий могут последовать далее?  
Опишите их.  

### Import libraries and Reading the data.

In [3]:
# Загрузка библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats

In [5]:
# Получение пути к файлам
catalog_path = "./data/Справочник.csv"
data_path = "./data/data.csv"

In [6]:
# Создание DF
catalog = pd.read_csv(catalog_path)
data = pd.read_csv(data_path)

### Review Data (Understanding the Data)

In [18]:
print(catalog.isnull().sum())
print(50 * "-")
print(data.isnull().sum())

Наименование филиала            0
Наименование региона филиала    0
Идентификатор филиала           0
dtype: int64
--------------------------------------------------
Идентификатор документа            0
Идентификатор товара               0
Вид операции документа             0
Дата документа                     0
Идентификатор филиалы документа    0
dtype: int64


In [46]:
data['Вид операции документа'].value_counts()

Прием товара     1499785
Выдача товара    1416812
Name: Вид операции документа, dtype: int64

In [50]:
data['Дата документа'].value_counts()

2019-11-27    8393
2019-10-27    8350
2019-11-08    8348
2019-07-10    8347
2019-03-23    8345
              ... 
2019-01-04    4353
2019-01-03    4270
2019-01-02    4185
2020-01-01    4109
2019-01-01    4109
Name: Дата документа, Length: 366, dtype: int64